In [ ]:
import IMAS
import FuseExchangeProtocol as FXP
import IJulia

if true
    # if using Redis via docker
    REDIS_HOST = "localhost"
    REDIS_PORT = 55000
    REDIS_PASSWORD = "redispw"
else
    REDIS_HOST = "redis-19689.c281.us-east-1-2.ec2.cloud.redislabs.com"
    REDIS_PORT = 19689
    REDIS_PASSWORD = ""
end;

subscription_client = FXP.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);

In [ ]:
"""
    service_ip_control(client::FXP.Client, session_id::String, service_name::String; timeout::Float64=10.0)

Function for servicing loop voltage control to achieve a desired Ip

After timeout of inactivity the service will be considered concluded and the session closed
"""
function service_ip_control(client::FXP.Client, session_id::String, service_name::String; timeout::Float64=10.0)
    # define a IMAS PID controller
    controller = IMAS.controllers__linear_controller()
    controller.name = "ip"
    
    while true
        # pop inputs
        inputs = FXP.json_pop(client, session_id, service_name, :provider; timeout, error_on_timeout=false)
        if inputs === nothing
            @info("DONE: $(session_id) $(service_name)")
            break
        end
        inputs = NamedTuple(inputs)

        # initialize controller with given PID coefficients
        if ismissing(controller.outputs, :data)
            IMAS.pid_controller(controller, inputs.P, inputs.I, inputs.D)
        end

        # run controller
        control = controller(inputs.setpoint, inputs.value, inputs.time)
        
        # push output
        FXP.json_push(client, session_id, service_name, :provider; control)
    end
end;

In [ ]:
# automated spawn
subscription_client = FXP.register_service(subscription_client, "ip_control", service_ip_control);

In [ ]:
# manual spawn
FXP.register_service(subscription_client, "ip_control", x->println(x))

In [ ]:
service_ip_control("8136311451093097212", "ip_control")